## Written Questions for Assignment 2

##### (a)

$$- \displaystyle\sum_{w \in Vocab}~y_wlog(\hat{y_w})= - \displaystyle\sum_{w \in Vocab~and~y_w=1}~y_wlog(\hat{y_w})=-log(\hat{y_o})$$

##### (2) negative-sampling loss function

negative-sampling方法在corpus中随机选取非前后文窗口中的词作为负样例来计算损失函数。这种方法的优点是它不需要损失函数在corpus中每一个词上的损失，对计算能力的要求会大大降低。

negative-sampling的损失函数是，

$$J_{neg-sampling}(v_c, o, U)=-log(\sigma(u_o^Tv_c))-\displaystyle\sum^{K}_{k=1}log(\sigma(-u_k^Tv_c))$$

其实，$\sigma(x)=\frac{1}{1+e^{-x}}$

为实践基于这一损失函数的模型训练，我计算出了$J_{neg_sampling}$对于$v_c$和$U$的偏导数，

$$\frac{\partial J_{neg-sampling}(v_c, o, U)}{\partial v_c}=- \frac{u_o}{e^{u_o^Tv_c}+1}+\displaystyle\sum^{K}_{k=1} \frac{u_k}{e^{-u_o^Tv_c}+1}$$

$$\left. \frac{\partial J_{neg-sampling}(v_c, o, U)}{\partial u}\right|_{u=u_o} =-\frac{v_c}{e^{u_o^Tv_c}+1}$$

$$\left. \frac{\partial J_{neg-sampling}(v_c, o, U)}{\partial u}\right|_{u\neq u_o} = \displaystyle\sum^{K}_{k=1}  \frac{v_c}{e^{-u_o^Tv_c}+1}$$

##### (1) naive-softmax loss function

naive-softamax的损失函数是，

$$J_{naive-softmax}(v_c, o, U)=-logP(O=o|C=c)$$


且证明可得，naive-softmax损失函数得到的损失值和cross-entropy损失函数得到的损失值是一样的，

$$cross~entropy=- \sum_{w\in Vocab}^{} y_wlog(\hat{y_w}) = -log(\hat{y_w}) = -logP(O=o|C=c)$$

为实践基于这一损失函数的模型训练，我计算出了$J_{naive-softmax}$对于$v_c$和$U$的偏导数，

$$\frac{\partial J_{naive-softmax}(v_c, o, U)}{\partial v_c}=-u_o^T +\sum_{w\in Vocab}P(O=o|C=c) u_w$$

$$\left. \frac{\partial J_{naive-softmax}(v_c, o, U)}{\partial u}\right|_{u=u_o} =v_c +P(O=o|C=c)v_c $$

$$\left. \frac{\partial J_{naive-softmax}(v_c, o, U)}{\partial u}\right|_{u\neq u_o} = \sum_{w\in Vocab~and~w\neq o} P(O=o|C=c)v_c $$

##### skip-gram算法

假设中心词是$c=w_t$，前后文窗口是$[w_{t-m}, \dots, w_{t-1},w_t,w_{t+1},\dots,w_{t+m}]$(`window_size`=m)。skip-gram对总损失的计算公式是，

$$J_{skip-gram}(v_c, o, U)=\displaystyle\sum_{-m\leq j \leq m~j\neq t} J(v_c, w_{t+j}, U)$$

等式右侧的$J(v_c, o, U)$可以是$J_{neg-sampling}(v_c, o, U)$，也可以是$J_{navie-softmax}(v_c, o, U)$。

因此，skip-gram损失函数对于$v_c$和$U$的偏导数是，

$$\frac{\partial J_{skip-gram}(v_c, w_{t+j}, U)}{\partial U} = \displaystyle\sum_{-m\leq j \leq m~j\neq t} \frac{\partial J(v_c, w_{t+j}, U)}{\partial U}$$

$$\frac{\partial J_{skip-gram}(v_c, w_{t+j}, U)}{\partial v_c} = \displaystyle\sum_{-m\leq j \leq m~j\neq t} \frac{\partial J(v_c, w_{t+j}, U)}{\partial v_c}$$

$$\left. \frac{\partial J_{skip-gram}(v_c, o, U)}{\partial v_w}\right|_{w\neq c} =0$$

##### 使用SGD训练模型

SGD的算法可描述为，

while J > threshold:
<br>&emsp;&emsp;window = sample_window(corpus)
<br>&emsp;&emsp;theta_gradient = evaluate_gradient(J, window, theta)
<br>&emsp;&emsp;theta = theta - alpha*theta_gradient

结合以上全部计算结果，即可对word2vec(skip-gram)模型进行训练。

##### 代码实践

对于该算法的代码实践详见 [word2vec code](files/HW2的副本)。

### 2. Neural Transition-Based Dependency Parser模型的实践（还未全部完成）

Dependency Parser是分析确定句子中头词（head）与其他词语之间的关系的模型。一个 Neural Transition-Based Dependency Parser模型有两部分组成：
1. Parser: 根据模型的判断建立词语之间的dependices，从而确定一个句子的dependency tree。
2. Neural Network: 通过大量数据训练从而作出词语关系的判断的神经网络。

##### Parser

此模型中使用的方法是transitioned-based parser，因此在每一步都会建立一个新的parse。整个模型的结构分为三块，分别是：
1. 一个stack：储存正在被处理的词语
2. 一个buffer： 储存未来要被处理的词语
3. 一个dependencies list： 记录模型预测的dependencies。

在初始状态下，stack中只有ROOT一个元素，而buffer存储了需要被parse的句子中的所有词语（根据在句子中的顺序），dependencies中此时没有任何内容。

Parse的每一步称为partial pars，可能是以下三种操作中的一种：

START: $\sigma=[ROOT],~\beta=w_1,\dots,w_n,~A=\emptyset$

1. `SHIFT`: 将buffer中的第一个词移位到stack中，$\sigma~~w_i|\beta~~A\Rightarrow \sigma|w_i~~\beta~~A$
2. `LEFT-ARC`: 将stack中第二新加的词语标记为第一新加的词语的关联，并在dependencies中加入第一新加的词$\rightarrow$第二新加词，将第二新加词从stack中移除，$\sigma｜w_i|w_j~~\beta~~A\Rightarrow \sigma|w_j~~\beta~~A\cup {r(w_j~w_i)}$
3. `RIGHT-ARC`: 将stack中第一新加的词语标记为第二新加的词语的关联，并在dependencies中加入第二新加的词$\rightarrow$第一新加词，将第一新加词从stack中移除，$\sigma｜w_i|w_j~~\beta~~A\Rightarrow \sigma|w_i~~\beta~~A\cup {r(w_i~w_j)}$

##### Neural Network

Neural Network的特征向量由由stack中的词以及buffer中的第一个词组成，表示为$W=[w_1,...,w_n]$，每个$0\leq m \geq |V|$是词语在词典中的index。NN随即赋予每一个词语一个embedding并合并在一起合成input向量，

$$X=[E_{w_1},\dots,E_{w_m}] \in {\Bbb R}^{dm}$$

其中$E\in{\Bbb R}^{|V|\times d}$是embedding矩阵

然后计算模型的预测结果，

$$h=ReLU(xW+b_1)$$

$$l=hU+b_2$$

$$\hat{y} = softmax(l)$$

其中$h$是神经网络中的hidden layer，$l$为logits，$\hat{y}$为最后的预测结果。训练NN，使用back-propogation来最小化cross-entropy损失函数，

$$J(\theta)=CE(y,\hat{y})=-\displaystyle\sum_{i=1}^{3} y_ilog \hat{y_i}$$

##### 代码实践

对于该算法的代码实践详见[neural transition-based dependency parser code](files/HW3的副本)（目前代码只完成了parser部分，NN部分还没有完成）。

### 3. 对基础Recurrent Neural Nework (RNN)结构相关理论的学习

对基础的RNN结构进行了了解和学习。计划在未来的学习中学习更加复杂的RNN结构，如GRU，LSTM，Multi-layer RNN。